In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
# Defining directories
dir_path = Path.cwd().parent
prep_folder = dir_path.joinpath("data/prep")
s1_path = prep_folder.joinpath("sent1.parquet")

processed_folder = dir_path.joinpath("data/processed")
if not processed_folder.exists():
    processed_folder.mkdir(parents=True)

In [3]:
df = pd.read_parquet(s1_path)
df

,date,long,lat,species_names,vv_Descending,vh_Descending,vv_Ascending,vh_Ascending
0,2019-08-31,75.66669580000007,32.25107380000003,Bambusoideae spp.,244.0,178.0,NaN,NaN
1,2019-08-31,75.66676390000003,32.25073910000003,Bambusoideae spp.,324.0,129.0,NaN,NaN
2,2019-08-31,75.66686810000004,32.25026770000005,Bambusoideae spp.,168.0,109.0,NaN,NaN
3,2019-08-31,75.66719240000003,32.249825100000066,Bambusoideae spp.,257.0,123.0,NaN,NaN
4,2019-08-31,75.66729090000007,32.24962370000003,Bambusoideae spp.,224.0,87.0,NaN,NaN
...,...,...,...,...,...,...,...,...
77813186,2021-11-19,76.46739500000007,32.928548300000045,Cedrus deodara,44.0,41.0,NaN,NaN
77813187,2021-11-19,76.46765970000007,32.92851550000006,Cedrus deodara,43.0,26.0,NaN,NaN
77813188,2021-11-19,76.46798620000004,32.93168870000005,Cedrus deodara,73.0,41.0,NaN,NaN
77813189,2021-11-19,76.46884810000006,32.93124820000003,Cedrus deodara,88.0,45.0,NaN,NaN


In [4]:
print(
    df['vh_Ascending'].isna().value_counts(),
df['vv_Ascending'].isna().value_counts(),
df['vh_Descending'].isna().value_counts(),
df['vv_Descending'].isna().value_counts()
)

vh_Ascending
False    42127713
True     35685478
Name: count, dtype: int64 vv_Ascending
False    42127723
True     35685468
Name: count, dtype: int64 vh_Descending
True     42169016
False    35644175
Name: count, dtype: int64 vv_Descending
True     42127736
False    35685455
Name: count, dtype: int64


In [5]:
#scaling fpr sentinel 1 
for band in ["vh_Ascending", "vv_Ascending", "vh_Descending", "vv_Descending"]:
    df[band] = 10 * np.log10(df[band])

# Extracting month and year
df["year"] = pd.to_datetime(df["date"]).dt.year
df["month"] = pd.to_datetime(df["date"]).dt.month
df.drop('date', axis=1, inplace=True)


In [6]:
print(
    df['vh_Ascending'].isna().value_counts(),
df['vv_Ascending'].isna().value_counts(),
df['vh_Descending'].isna().value_counts(),
df['vv_Descending'].isna().value_counts()
)

vh_Ascending
False    42127713
True     35685478
Name: count, dtype: int64 vv_Ascending
False    42127723
True     35685468
Name: count, dtype: int64 vh_Descending
True     42169016
False    35644175
Name: count, dtype: int64 vv_Descending
True     42127736
False    35685455
Name: count, dtype: int64


In [6]:
def make_s1_indices(df):
    #### SENTINEL 
    ## VV VH Ratio
    df['VV_VH_A'] = df['vv_Ascending'] / df['vh_Ascending']
    df['VV_VH_D'] = df['vv_Descending'] / df['vh_Descending']

    ## VH VV Ratio
    df['VH_VV_A'] = df['vh_Ascending'] / df['vv_Ascending']
    df['VH_VV_D'] = df['vh_Descending'] / df['vv_Descending']

    ## SAR Normalized Difference Vegetation Index 
    df['SAR_NDVI_A'] = (df['vh_Ascending'] - df['vv_Ascending']) / (df['vh_Ascending'] + df['vv_Ascending'])
    df['SAR_NDVI_D'] = (df['vh_Descending'] - df['vv_Descending']) / (df['vh_Descending'] + df['vv_Descending'])

    ## Difference Vegetation Index
    df['DVI_A'] = df['vh_Ascending'] - df['vv_Ascending']
    df['DVI_D'] = df['vh_Descending'] - df['vv_Descending']

    ## Sum Vegetation Index
    df['SVI_A'] = df['vh_Ascending'] + df['vv_Ascending']
    df['SVI_D'] = df['vh_Descending'] + df['vv_Descending']

    ## Relative Difference Vegetation Index (RDVI)
    df['RDVI_A'] = df['VH_VV_A'] - df['VV_VH_A']
    df['RDVI_D'] = df['VH_VV_D'] - df['VV_VH_D']

    ## Normalized Relative Difference Vegetation Index
    df['NRDVI_A'] = df['RDVI_A'] / (df['VH_VV_A'] + df['VV_VH_A'])
    df['NRDVI_D'] = df['RDVI_D'] / (df['VH_VV_D'] + df['VV_VH_D'])

    ## Sum of Squared Difference Vegetation Indexsrc/make_model_df.ipynb
    df['SSDVI_A'] = df['vh_Ascending'].pow(2) - df['vv_Ascending'].pow(2)
    df['SSDVI_D'] = df['vh_Descending'].pow(2) - df['vv_Descending'].pow(2)

    return df

df_indices = make_s1_indices(df)
df_indices

,long,lat,species_names,vv_Descending,vh_Descending,vv_Ascending,vh_Ascending,year,month,VV_VH_A,...,DVI_A,DVI_D,SVI_A,SVI_D,RDVI_A,RDVI_D,NRDVI_A,NRDVI_D,SSDVI_A,SSDVI_D
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,23.873898,22.504200,NaN,NaN,2019,8,NaN,...,NaN,-1.369698,NaN,46.378098,NaN,-0.118236,NaN,-0.059015,NaN,-63.524000
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,25.105450,21.105897,NaN,NaN,2019,8,NaN,...,NaN,-3.999553,NaN,46.211347,NaN,-0.348809,NaN,-0.171811,NaN,-184.824732
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,22.253093,20.374265,NaN,NaN,2019,8,NaN,...,NaN,-1.878828,NaN,42.627358,NaN,-0.176646,NaN,-0.087980,NaN,-80.089466
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,24.099331,20.899051,NaN,NaN,2019,8,NaN,...,NaN,-3.200280,NaN,44.998382,NaN,-0.285926,NaN,-0.141524,NaN,-144.007428
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,23.502480,19.395193,NaN,NaN,2019,8,NaN,...,NaN,-4.107288,NaN,42.897673,NaN,-0.386528,NaN,-0.189753,NaN,-176.193082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77813186,76.46739500000007,32.928548300000045,Cedrus deodara,16.434527,16.127839,NaN,NaN,2021,11,NaN,...,NaN,-0.306688,NaN,32.562365,NaN,-0.037677,NaN,-0.018835,NaN,-9.986493
77813187,76.46765970000007,32.92851550000006,Cedrus deodara,16.334685,14.149733,NaN,NaN,2021,11,NaN,...,NaN,-2.184951,NaN,30.484418,NaN,-0.288178,NaN,-0.142616,NaN,-66.606962
77813188,76.46798620000004,32.93168870000005,Cedrus deodara,18.633229,16.127839,NaN,NaN,2021,11,NaN,...,NaN,-2.505390,NaN,34.761067,NaN,-0.289804,NaN,-0.143404,NaN,-87.090031
77813189,76.46884810000006,32.93124820000003,Cedrus deodara,19.444827,16.532125,NaN,NaN,2021,11,NaN,...,NaN,-2.912702,NaN,35.976952,NaN,-0.325977,NaN,-0.160866,NaN,-104.790125


### Making Monthly Medians

In [7]:
index_cols = ["long", "lat", "year", "month", "species_names"]
#making monthlu medians
df_month_medians = (
    df_indices.groupby(index_cols)
    .median()
    .reset_index()
)

df_month_medians

,long,lat,year,month,species_names,vv_Descending,vh_Descending,vv_Ascending,vh_Ascending,VV_VH_A,...,DVI_A,DVI_D,SVI_A,SVI_D,RDVI_A,RDVI_D,NRDVI_A,NRDVI_D,SSDVI_A,SSDVI_D
0,75.66669580000007,32.25107380000003,2018,1,Bambusoideae spp.,23.174552,19.014211,22.405492,19.777236,1.104054,...,-2.111655,-4.160341,42.699330,42.188762,-0.198301,-0.398812,-0.098667,-0.195303,-90.166242,-175.427345
1,75.66669580000007,32.25107380000003,2018,2,Bambusoideae spp.,23.242825,19.956352,22.166288,21.461723,1.033557,...,-0.704565,-3.694342,43.628012,43.371197,-0.065693,-0.344880,-0.032812,-0.169932,-30.259764,-156.206808
2,75.66669580000007,32.25107380000003,2018,3,Bambusoideae spp.,21.281183,18.450980,23.676237,20.093502,1.179084,...,-3.582735,-2.830202,43.769740,39.732163,-0.330660,-0.291276,-0.163029,-0.143803,-156.466199,-111.066850
3,75.66669580000007,32.25107380000003,2018,4,Bambusoideae spp.,22.552725,19.867717,23.384565,19.956352,1.212000,...,-4.090376,-2.337480,44.595282,41.528996,-0.386918,-0.218054,-0.189937,-0.108385,-174.572138,-100.525548
4,75.66669580000007,32.25107380000003,2018,5,Bambusoideae spp.,22.848040,19.700091,22.878017,19.956352,1.116375,...,-2.467820,-3.147949,41.451342,42.548131,-0.220619,-0.303061,-0.109644,-0.149384,-110.754026,-132.122432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12508183,78.12624330000006,31.258713300000068,2024,8,Rhododendron arboreum,23.092960,19.633255,22.068678,21.522453,1.025432,...,-0.546225,-2.763381,43.591130,43.201884,-0.049914,-0.255674,-0.024937,-0.126749,-23.920265,-119.981582
12508184,78.12624330000006,31.258713300000068,2024,9,Rhododendron arboreum,19.493900,19.030900,25.024271,21.105897,1.156163,...,-3.446241,-2.238640,44.801507,36.569602,-0.291233,-0.250365,-0.144097,-0.124213,-163.981665,-78.146481
12508185,78.12624330000006,31.258713300000068,2024,10,Rhododendron arboreum,17.960884,17.548700,22.818354,20.740081,1.100403,...,-2.078273,-0.230907,43.558436,35.348660,-0.191551,-0.024033,-0.095325,-0.012014,-90.421479,-8.881958
12508186,78.12624330000006,31.258713300000068,2024,11,Rhododendron arboreum,18.920946,16.720979,22.174839,19.344985,1.129331,...,-2.461291,-2.199967,41.519824,35.641925,-0.243852,-0.247841,-0.121029,-0.122980,-105.804204,-78.411074


### Making Season Medians 

In [8]:
# creating seasons
season_index_cols = ["long", "lat", "season", "species_names"]


conds = [df_month_medians["month"].isin([3, 4, 5]), df_month_medians["month"].isin([10, 11, 12])]
opts = ["summer", "winter"]


df_month_medians["season"] = np.select(conds, opts, default="NA")
df_season_medians = df_month_medians[~(df_month_medians["season"] == "NA")]
df_season_medians["season"] = df_season_medians["season"] + df_month_medians["year"].astype(str) 


df_season_medians = df_season_medians.groupby(season_index_cols).median().reset_index()
df_season_medians

/tmp/ipykernel_1526362/2127264070.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_season_medians["season"] = df_season_medians["season"] + df_month_medians["year"].astype(str)


,long,lat,season,species_names,year,month,vv_Descending,vh_Descending,vv_Ascending,vh_Ascending,...,DVI_A,DVI_D,SVI_A,SVI_D,RDVI_A,RDVI_D,NRDVI_A,NRDVI_D,SSDVI_A,SSDVI_D
0,75.66669580000007,32.25107380000003,summer2018,Bambusoideae spp.,2018.0,4.0,22.552725,19.700091,23.384565,19.956352,...,-3.582735,-2.830202,43.769740,41.528996,-0.330660,-0.291276,-0.163029,-0.143803,-156.466199,-111.066850
1,75.66669580000007,32.25107380000003,summer2019,Bambusoideae spp.,2019.0,4.0,23.856063,19.956352,21.846914,20.530784,...,-1.316130,-3.943802,42.433853,43.768323,-0.124348,-0.363376,-0.062054,-0.178761,-55.774555,-172.613601
2,75.66669580000007,32.25107380000003,summer2020,Bambusoideae spp.,2020.0,4.0,23.268281,21.227070,22.068259,20.086002,...,-2.090337,-4.028260,42.235219,43.871136,-0.199344,-0.360037,-0.099106,-0.177096,-87.990981,-171.256533
3,75.66669580000007,32.25107380000003,summer2021,Bambusoideae spp.,2021.0,4.0,24.927604,19.777236,22.430380,20.293838,...,-2.785947,-4.474374,42.724218,44.478554,-0.253907,-0.398256,-0.125368,-0.195294,-123.071242,-203.050827
4,75.66669580000007,32.25107380000003,summer2022,Bambusoideae spp.,2022.0,4.0,24.149733,20.453230,23.242825,20.487848,...,-2.469331,-3.876463,44.256146,44.533811,-0.228456,-0.348859,-0.113490,-0.171835,-107.109181,-173.599348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084693,78.12624330000006,31.258713300000068,winter2020,Rhododendron arboreum,2020.0,11.0,21.895621,19.714503,23.283796,19.235085,...,-4.198946,-2.402251,42.757340,40.691233,-0.400352,-0.231446,-0.196282,-0.114956,-177.903651,-98.577484
2084694,78.12624330000006,31.258713300000068,winter2021,Rhododendron arboreum,2021.0,11.0,23.242825,21.789769,23.443923,20.043214,...,-3.553042,-2.485054,43.429704,45.032594,-0.334547,-0.259086,-0.164246,-0.128468,-152.540861,-103.770884
2084695,78.12624330000006,31.258713300000068,winter2022,Rhododendron arboreum,2022.0,11.0,20.606978,17.320957,22.648178,21.522883,...,-1.395479,-2.559417,44.171062,37.472564,-0.123384,-0.276403,-0.061530,-0.136900,-63.214959,-97.142729
2084696,78.12624330000006,31.258713300000068,winter2023,Rhododendron arboreum,2023.0,11.0,20.718820,16.989700,22.944662,20.334238,...,-2.808266,-1.894584,43.711418,37.708520,-0.264081,-0.197352,-0.130904,-0.098199,-119.971852,-64.619521


There are no NaNs in the resulting frame because its created from monthly medians

### Splitting Testing and Training
All data from 2024 will be assigned for testing and data from 2018 - 2023 (6 years) will be used for training.

In [9]:
sent1_train_season_medians = df_season_medians[df_season_medians['year']!=2024]
sent1_test_season_medians = df_season_medians[df_season_medians['year']==2024]


sent1_train_season_medians = sent1_train_season_medians.drop(["year", "month"], axis=1)
sent1_test_season_medians = sent1_test_season_medians.drop(["year", "month"], axis=1)

sent1_train_season_medians

,long,lat,season,species_names,vv_Descending,vh_Descending,vv_Ascending,vh_Ascending,VV_VH_A,VV_VH_D,...,DVI_A,DVI_D,SVI_A,SVI_D,RDVI_A,RDVI_D,NRDVI_A,NRDVI_D,SSDVI_A,SSDVI_D
0,75.66669580000007,32.25107380000003,summer2018,Bambusoideae spp.,22.552725,19.700091,23.384565,19.956352,1.179084,1.156924,...,-3.582735,-2.830202,43.769740,41.528996,-0.330660,-0.291276,-0.163029,-0.143803,-156.466199,-111.066850
1,75.66669580000007,32.25107380000003,summer2019,Bambusoideae spp.,23.856063,19.956352,21.846914,20.530784,1.064105,1.198059,...,-1.316130,-3.943802,42.433853,43.768323,-0.124348,-0.363376,-0.062054,-0.178761,-55.774555,-172.613601
2,75.66669580000007,32.25107380000003,summer2020,Bambusoideae spp.,23.268281,21.227070,22.068259,20.086002,1.104880,1.196235,...,-2.090337,-4.028260,42.235219,43.871136,-0.199344,-0.360037,-0.099106,-0.177096,-87.990981,-171.256533
3,75.66669580000007,32.25107380000003,summer2021,Bambusoideae spp.,24.927604,19.777236,22.430380,20.293838,1.136516,1.218761,...,-2.785947,-4.474374,42.724218,44.478554,-0.253907,-0.398256,-0.125368,-0.195294,-123.071242,-203.050827
4,75.66669580000007,32.25107380000003,summer2022,Bambusoideae spp.,24.149733,20.453230,23.242825,20.487848,1.120731,1.189528,...,-2.469331,-3.876463,44.256146,44.533811,-0.228456,-0.348859,-0.113490,-0.171835,-107.109181,-173.599348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084692,78.12624330000006,31.258713300000068,winter2019,Rhododendron arboreum,21.450173,18.317562,26.085735,23.273577,1.127325,1.171387,...,-2.812157,-3.046949,49.422264,38.482874,-0.230103,-0.317698,-0.112385,-0.156882,-139.104165,-136.152344
2084693,78.12624330000006,31.258713300000068,winter2020,Rhododendron arboreum,21.895621,19.714503,23.283796,19.235085,1.220015,1.122397,...,-4.198946,-2.402251,42.757340,40.691233,-0.400352,-0.231446,-0.196282,-0.114956,-177.903651,-98.577484
2084694,78.12624330000006,31.258713300000068,winter2021,Rhododendron arboreum,23.242825,21.789769,23.443923,20.043214,1.182678,1.137903,...,-3.553042,-2.485054,43.429704,45.032594,-0.334547,-0.259086,-0.164246,-0.128468,-152.540861,-103.770884
2084695,78.12624330000006,31.258713300000068,winter2022,Rhododendron arboreum,20.606978,17.320957,22.648178,21.522883,1.063837,1.147706,...,-1.395479,-2.559417,44.171062,37.472564,-0.123384,-0.276403,-0.061530,-0.136900,-63.214959,-97.142729


### Widening the frame

#### Widening Train

In [10]:
sent1_train_season_medians_wide = sent1_train_season_medians.pivot(index=['long', 'lat', 'species_names'], columns=['season']).reset_index()
new_cols = ["".join(x) for x in sent1_train_season_medians_wide.columns]
sent1_train_season_medians_wide.columns = new_cols
sent1_train_season_medians_wide

,long,lat,species_names,vv_Descendingsummer2018,vv_Descendingsummer2019,vv_Descendingsummer2020,vv_Descendingsummer2021,vv_Descendingsummer2022,vv_Descendingsummer2023,vv_Descendingwinter2018,...,SSDVI_Dsummer2020,SSDVI_Dsummer2021,SSDVI_Dsummer2022,SSDVI_Dsummer2023,SSDVI_Dwinter2018,SSDVI_Dwinter2019,SSDVI_Dwinter2020,SSDVI_Dwinter2021,SSDVI_Dwinter2022,SSDVI_Dwinter2023
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,22.552725,23.856063,23.268281,24.927604,24.149733,22.988531,23.639657,...,-171.256533,-203.050827,-173.599348,-110.127719,-107.537649,-103.918188,-163.082568,-157.318045,-107.662974,-126.133744
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,23.024369,22.741578,22.165247,22.304489,23.159703,23.443923,22.861569,...,-98.898624,-135.881860,-198.235266,-190.950782,-75.833022,-167.804041,-100.996840,-93.238388,-171.721775,-92.572046
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,22.648178,21.367206,23.846887,23.673559,23.692159,24.082400,23.548257,...,-153.618089,-91.995077,-98.083866,-153.763294,-87.641060,-126.581235,-119.597271,-153.079170,-92.642563,-158.337847
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,22.492587,23.747483,24.486953,22.638665,22.703899,23.053514,23.234566,...,-188.052899,-122.337909,-99.774765,-91.761333,-135.078548,-172.031322,-148.356240,-94.093636,-139.379619,-167.461496
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,22.454005,22.624511,23.603979,22.789580,22.407571,24.983106,22.430380,...,-176.305449,-94.742753,-118.860856,-175.866812,-113.844266,-115.665871,-129.297871,-62.714319,-120.826652,-135.604237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148902,78.12497830000007,31.25865170000003,Rhododendron arboreum,20.211893,21.731863,18.750613,16.901961,16.532125,19.590414,20.679478,...,-39.702539,-49.188099,-54.201908,-101.887123,-91.712916,-93.879162,-62.260566,-44.825354,-84.197495,-63.260309
148903,78.12522000000007,31.258880000000033,Rhododendron arboreum,22.227165,20.334238,18.750613,18.388491,17.708520,17.437106,22.174839,...,-66.233937,-120.758560,-67.567300,-49.554456,-115.389311,-117.603491,-78.242384,-140.258813,-44.242597,-30.490808
148904,78.12614830000007,31.258660000000077,Rhododendron arboreum,17.160033,25.717088,20.827854,19.242793,19.242793,18.885684,24.683473,...,-112.532693,-87.908529,-81.635170,-72.044555,-98.481586,-103.935521,-73.605322,-117.258229,-60.332410,-62.057653
148905,78.12618260000005,31.25882800000005,Rhododendron arboreum,17.160033,25.404557,21.105897,22.013971,20.170333,18.162286,24.857214,...,-97.153136,-122.439780,-115.137414,-60.630459,-103.198947,-93.814746,-116.910815,-139.357466,-74.009928,-42.494588


##### Widen Test

In [11]:
sent1_test_season_medians_wide = sent1_test_season_medians.pivot(index=['long', 'lat', 'species_names'], columns=['season']).reset_index()
new_cols = ["".join(x) for x in sent1_test_season_medians_wide.columns]
sent1_test_season_medians_wide.columns = new_cols
sent1_test_season_medians_wide

,long,lat,species_names,vv_Descendingsummer2024,vv_Descendingwinter2024,vh_Descendingsummer2024,vh_Descendingwinter2024,vv_Ascendingsummer2024,vv_Ascendingwinter2024,vh_Ascendingsummer2024,...,RDVI_Dsummer2024,RDVI_Dwinter2024,NRDVI_Asummer2024,NRDVI_Awinter2024,NRDVI_Dsummer2024,NRDVI_Dwinter2024,SSDVI_Asummer2024,SSDVI_Awinter2024,SSDVI_Dsummer2024,SSDVI_Dwinter2024
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,22.329961,23.283796,20.855999,18.947904,22.922561,23.829795,20.844843,...,-0.199947,-0.321819,-0.127756,-0.165662,-0.099478,-0.158866,-137.101837,-163.194938,-98.101772,-150.438419
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,23.710679,23.786145,19.777236,19.955244,20.844843,19.280017,16.127839,...,-0.377909,-0.266552,-0.238637,-0.106794,-0.185669,-0.132108,-170.728100,-71.635497,-182.401575,-133.415633
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,24.082400,23.820170,21.139434,21.449505,22.718416,23.069685,20.469209,...,-0.262839,-0.236903,-0.077496,-0.099564,-0.130299,-0.117629,-76.977260,-95.889959,-133.902116,-116.844349
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,21.702617,23.463530,20.713199,20.934217,23.260264,23.891661,22.001567,...,-0.171177,-0.199373,-0.056007,-0.100159,-0.085277,-0.099195,-55.821320,-104.322354,-72.590927,-99.906539
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,25.352941,23.563249,22.088770,21.461280,22.510769,23.018103,21.205615,...,-0.186049,-0.221375,-0.118311,-0.118761,-0.092069,-0.110016,-119.092017,-109.853732,-99.370167,-105.925533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148902,78.12497830000007,31.25865170000003,Rhododendron arboreum,18.888946,18.692317,16.334685,17.132556,23.207470,22.411509,21.139434,...,-0.170429,-0.181020,-0.083127,-0.092637,-0.084907,-0.090117,-82.164870,-88.577731,-59.254315,-55.170010
148903,78.12522000000007,31.258880000000033,Rhododendron arboreum,17.708520,17.923917,16.627578,16.213949,22.671717,23.344351,20.148947,...,-0.214484,-0.145798,-0.075293,-0.155308,-0.106631,-0.072706,-65.420747,-165.360064,-70.973345,-43.825039
148904,78.12614830000007,31.258660000000077,Rhododendron arboreum,19.411484,19.242793,17.198242,16.627578,23.424227,23.057820,20.936193,...,-0.287797,-0.191686,-0.090191,-0.089627,-0.142432,-0.094224,-90.786151,-87.431108,-93.516283,-82.358827
148905,78.12618260000005,31.25882800000005,Rhododendron arboreum,19.912261,18.384382,17.993405,17.428607,22.355284,21.180166,20.453230,...,-0.128113,-0.245241,-0.067839,-0.099945,-0.063926,-0.121709,-72.741127,-81.024611,-57.003766,-88.151803


In [12]:
for col in sent1_train_season_medians_wide.columns:
    print(pd.Series(sent1_train_season_medians_wide[col].isna().value_counts()))

long
False    148907
Name: count, dtype: int64
lat
False    148907
Name: count, dtype: int64
species_names
False    148907
Name: count, dtype: int64
vv_Descendingsummer2018
False    148907
Name: count, dtype: int64
vv_Descendingsummer2019
False    148907
Name: count, dtype: int64
vv_Descendingsummer2020
False    148907
Name: count, dtype: int64
vv_Descendingsummer2021
False    148907
Name: count, dtype: int64
vv_Descendingsummer2022
False    148907
Name: count, dtype: int64
vv_Descendingsummer2023
False    148907
Name: count, dtype: int64
vv_Descendingwinter2018
False    148907
Name: count, dtype: int64
vv_Descendingwinter2019
False    148907
Name: count, dtype: int64
vv_Descendingwinter2020
False    148907
Name: count, dtype: int64
vv_Descendingwinter2021
False    148907
Name: count, dtype: int64
vv_Descendingwinter2022
False    148907
Name: count, dtype: int64
vv_Descendingwinter2023
False    148907
Name: count, dtype: int64
vh_Descendingsummer2018
False    148907
Name: count, dtype:

### Exporting Frames

In [13]:
sent_1_month_medians_path = processed_folder.joinpath("sent_1_month_medians.parquet")
sent_1_season_medians_path = processed_folder.joinpath("sent_1_season_medians.parquet")

sent1_train_season_medians_widened_path = processed_folder.joinpath("sent1_train_season_medians_widened.parquet")
sent1_test_season_medians_widened_path = processed_folder.joinpath("sent1_test_season_medians_widened.parquet")

In [14]:
df_month_medians.to_parquet(sent_1_month_medians_path, index=False)
df_season_medians.to_parquet(sent_1_season_medians_path, index=False)

In [15]:
sent1_train_season_medians_wide.to_parquet(sent1_train_season_medians_widened_path, index=False)
sent1_test_season_medians_wide.to_parquet(sent1_test_season_medians_widened_path, index=False)